In [7]:
%load_ext sql
%sql sqlite:///pokemon.sqlite
from utils import *

In [ ]:
def view_trainadores(limite=None, mostrar=False):
    treinadores = Trainer.objects.all()
    if limite:
        treinadores = treinadores[:limite]
        
    render('trainers.html', context={'trainers': treinadores}, show=mostrar)

In [ ]:
# Visualizando o que a view retorna

view_trainadores(limite=3, mostrar=True)

In [2]:
Trainer.objects.count()

11860

#### Chronometrando seu código

``` python
import timeit as pytimeit

def timeit(func):
    total = pytimeit.timeit(func, number=1)
    print('Ran in:', total)
```

In [ ]:
timeit(view_trainadores)

# Investigando a quantidade de consultas ao Banco de Dados

In [4]:
# Objeto "connection"
from django.db import reset_queries
from django.db import connection

reset_queries()

print("Queries:", connection.queries)

Trainer.objects.count()

print("Queries:", connection.queries)

Queries: []
Queries: [{'sql': 'SELECT COUNT(*) AS "__count" FROM "Trainer"', 'time': '0.001'}]


#### Contando queries

``` python
class count_queries:
    def __init__(self):
        self.queries = []

    def __enter__(self):
        self.initial = len(connection.queries)
        return self

    def __exit__(self, *args, **kwargs):
        self.queries = [q['sql'] for q in connection.queries[self.initial:]]
        print('Number of queries:', len(self.queries))
```

In [ ]:
with count_queries():
    view_trainadores(limite=100)

In [8]:
display_template('trainers.html')

In [ ]:
with count_queries() as m:
    treinadores = Trainer.objects.all()[:100]
    
    for t in treinadores:
        t.favorite_pokemon.name
        
pretty(m.queries[:10])

In [ ]:
consulta_treinadores = m.queries[0]
print(consulta_treinadores)

In [ ]:
%sql $consulta_treinadores

In [ ]:
consulta_pokemon = m.queries[1]
print(consulta_pokemon)

In [ ]:
%sql $consulta_pokemon

In [ ]:
query = (
    'SELECT *'
    'FROM "Trainer" LEFT OUTER JOIN "PokemonKind"'
    'ON ("Trainer"."favorite_pokemon_id" = "PokemonKind"."id")'
    'LIMIT 100'
)
%sql $query

In [ ]:
with count_queries() as m:
    treinadores = Trainer.objects.select_related('favorite_pokemon').all()[:100]
    
    for t in treinadores:
        t.favorite_pokemon.name
        
print(m.queries[0])

In [ ]:
def view_trainadores(limite=None):
    treinadores = Trainer.objects.select_related("favorite_pokemon").all()
    
    if limite:
        treinadores = treinadores[:limite]
        
    render('trainers.html', context={'trainers': treinadores})
    

with count_queries():
    view_trainadores(limite=100)

In [ ]:
display_template('trainers.html')

In [ ]:
with count_queries() as m:
    treinadores = Trainer.objects.all()[:100]
    
    for t in treinadores:
        for p in t.pokemons.all():
            p.pokelevel

In [ ]:
Trainer.objects.select_related('pokemons').all()

In [ ]:
with count_queries() as m:
    treinadores = Trainer.objects.prefetch_related('pokemons').all()[:100]
    
    for t in treinadores:
        for p in t.pokemons.all():
            p.pokelevel

In [ ]:
consulta = m.queries[0]
print(consulta)

In [ ]:
consulta = m.queries[1]
print(consulta)

In [ ]:
def view_trainadores(limite=None):
    treinadores = (
        Trainer.objects
        .select_related("favorite_pokemon")
        .prefetch_related('pokemons')
        .all()
    )
    
    if limite:
        treinadores = treinadores[:limite]
        
    render('trainers.html', context={'trainers': treinadores})
    

with count_queries():
    view_trainadores(limite=100)

In [ ]:
display_template('trainers.html')

In [ ]:
with count_queries() as m:
    treinadores = (
        Trainer.objects
        .prefetch_related('pokemons')
        .all()[:100]
    )
    
    for t in treinadores:
        for p in t.pokemons.all():
            p.pokemon_kind.name

In [ ]:
with count_queries() as m:
    treinadores = (
        Trainer.objects
        .prefetch_related('pokemons', 'pokemons__pokemon_kind')
        .all()[:100]
    )
    
    for t in treinadores:
        for p in t.pokemons.all():
            p.pokemon_kind.name

# Testando as melhorias

In [ ]:
def view_trainadores(limite=None):
    treinadores = (
        Trainer.objects
        .select_related('favorite_pokemon')
        .prefetch_related('pokemons', 'pokemons__pokemon_kind')
        .all()
    )
        
    render('trainers.html', context={'trainers': treinadores})
    

with count_queries():
    view_trainadores()

In [ ]:
timeit(view_trainadores)

# Delegando o trabalho para o Banco de dados

In [ ]:
def calcular_medias():
    treinadores = (
        Trainer.objects
        .prefetch_related('pokemons')
        .all()
    )

    for treinador in treinadores:
        pokemons = treinador.pokemons.all()
        treinador.avg_level = (
            sum([p.pokelevel for p in pokemons]) 
            / len(pokemons)
        ) if pokemons else 0
        
    return treinadores


timeit(calcular_medias)

In [ ]:
from django.db.models import Avg

def calcular_medias():
    return list(
        Trainer.objects
        .annotate(avg_level=Avg('pokemons__pokelevel'))
        .all()
    )

treinadores = calcular_medias()

print(f'Treinador: {treinadores[0].name}')
print(f'Média de level: {treinadores[0].avg_level}')

In [ ]:
timeit(calcular_medias)

with count_queries() as m:
    treinadores = calcular_medias()

print(m.queries)
print(f'Treinador: {treinadores[0].name}')
print(f'Média de level: {treinadores[0].avg_level}')

# Escrevendo Testes

1. Instancie um objeto da forma mais completa possível (atenção para relacionamentos)
2. Teste o número de queries ao banco de dados.
3. Instancie um novo objeto.
4. Teste o número de queries ao banco de dados permanece o mesmo.

``` python
from django.test import TestCase

class TestView(TestCase):
    
    def test_number_of_queries(self):
        trainer1 = Trainer.objects.create(name="New")
        charizard = PokemonKind.objects.get(id=333)
        Pokemon.objects.create(
            trainer=trainer1, 
            pokemon_kind=charizard,
        )
        
        with self.assertNumQueries(2):
            calcular_medias()
            
        trainer2 = Trainer.objects.create(name="Other")
        bulbasaur = PokemonKind.objects.get(id=222)
        Pokemon.objects.create(
            trainer=trainer2, 
            pokemon_kind=bulbasaur,
        )        
            
        with self.assertNumQueries(2):
            calcular_medias()
```